# Capítulo 7 - Hipótese e inferência

In [8]:
from typing import Tuple
import math

## Exemplo: lançando uma moeda

In [9]:
def aproximacao_normal_para_binomial(n: int, p: float) -> Tuple[float, float]:
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def probabilidade_normal_acima(lo: float, mu: float = 0, sigma: float = 1) -> float:
    return 1 - normal_cdf(lo, mu, sigma)

def probabilidade_normal_intermediario(lo: float, hi: float, mu: float = 0, sigma: float = 1) -> float:
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

def probabilidade_normal_fora(lo: float, hi: float, mu: float = 0, sigma: float = 1) -> float:
    return 1 - probabilidade_normal_intermediario(lo, hi, mu, sigma)

In [10]:
def normal_inversa_cdf(p: float, mu: float = 0, sigma: float = 1, tolerancia: float = 0.00001) -> float:
    if mu != 0 or sigma != 1:
        return mu + sigma * normal_inversa_cdf(p, tolerancia = tolerancia)
    low_z = -10
    hi_z = 10
    while hi_z - low_z > tolerancia:
        mid_z = (low_z + hi_z) / 2
        mid_p = normal_cdf(mid_z)
        if mid_p < p:
            low_z = mid_z
        else:
            hi_z = mid_z
    return mid_z

def normal_limite_superior(probabilidade: float, mu: float = 0, sigma: float = 1) -> float:
    return normal_inversa_cdf(probabilidade, mu, sigma)

def normal_limite_inferior(probabilidade: float, mu: float = 0, sigma: float = 1) -> float:
    return normal_inversa_cdf(1 - probabilidade, mu, sigma)

def normal_dois_lados(probabilidade: float, mu: float = 0, sigma: float = 1) -> Tuple[float, float]:
    probabilidade_cauda = (1 - probabilidade) / 2
    limite_superior = normal_limite_inferior(probabilidade_cauda, mu, sigma)
    limite_inferior = normal_limite_superior(probabilidade_cauda, mu, sigma)
    return limite_inferior, limite_superior

In [11]:
mu_0, sigma_0 = aproximacao_normal_para_binomial(1000, 0.5)
print(mu_0, sigma_0)

500.0 15.811388300841896


In [13]:
limite_inferior, limite_superior = normal_dois_lados(0.95, mu_0, sigma_0)
print(limite_inferior, limite_superior)

469.01026640487555 530.9897335951244


In [14]:
lo, hi = normal_dois_lados(0.95, mu_0, sigma_0)
mu_1, sigma_1 = aproximacao_normal_para_binomial(1000, 0.55)
probabilidade_tipo_2 = probabilidade_normal_intermediario(lo, hi, mu_1, sigma_1)
power = 1 - probabilidade_tipo_2
power

0.886548001295367

In [15]:
hi = normal_limite_superior(0.95, mu_0, sigma_0)
hi

526.0073585242053

In [16]:
probabilidade_tipo_2 = normal_cdf(hi, mu_1, sigma_1)
power = 1 - probabilidade_tipo_2
power

0.9363794803307173

## P-valor